### Step 1 - Import Archives and Librarys

In [8]:
import pandas as pd
import pathlib as pl

In [2]:
emails = pd.read_excel(r'Databases/emails.xlsx')
stores = pd.read_csv(r'Databases/stores.csv', encoding='latin-1', sep=';')   # Using latin-1 encoding because the file has some characteres not recognized by utf-8
sales = pd.read_excel(r'Databases/sales.xlsx')

### Step 2 - Define one sheet to each store and definne the indicator day


In [3]:
# include store name to "sales"

sales = sales.merge(stores, on='Store ID')

,Sale Code,Date,Store ID,Product,Quantity,Unit Value,Final Value,Store
0,1,2019-01-01,1,Sapato Estampa,1,358,358,Iguatemi Esplanada
1,1,2019-01-01,1,Camiseta,2,180,360,Iguatemi Esplanada
2,1,2019-01-01,1,Sapato Xadrez,1,368,368,Iguatemi Esplanada
3,2,2019-01-02,3,Relógio,3,200,600,Norte Shopping
4,2,2019-01-02,3,Chinelo Liso,1,71,71,Norte Shopping
...,...,...,...,...,...,...,...,...
100994,69996,2019-12-26,17,Short Listrado,2,102,204,Center Shopping Uberlândia
100995,69996,2019-12-26,17,Mochila,4,270,1080,Center Shopping Uberlândia
100996,69996,2019-12-26,17,Pulseira Estampa,1,87,87,Center Shopping Uberlândia
100997,69997,2019-12-26,11,Camisa Listrado,1,108,108,Ribeirão Shopping


"for item in stores['Store']:\n    store_dictionary[item] = sales.loc[sales['Store']==item, :]\n\ndisplay(store_dictionary)"

In [5]:
store_dictionary = {}

for item in stores['Store']:
    store_dictionary[item] = sales.loc[sales['Store']==item, :]

display(store_dictionary['Iguatemi Esplanada'])
display(store_dictionary['Shopping Eldorado'])

,Sale Code,Date,Store ID,Product,Quantity,Unit Value,Final Value,Store
0,1,2019-01-01,1,Sapato Estampa,1,358,358,Iguatemi Esplanada
1,1,2019-01-01,1,Camiseta,2,180,360,Iguatemi Esplanada
2,1,2019-01-01,1,Sapato Xadrez,1,368,368,Iguatemi Esplanada
26,21,2019-01-02,1,Camisa Gola V Listrado,2,116,232,Iguatemi Esplanada
44,34,2019-01-02,1,Sapato Listrado,1,363,363,Iguatemi Esplanada
...,...,...,...,...,...,...,...,...
100847,69899,2019-12-25,1,Sapato Xadrez,4,368,1472,Iguatemi Esplanada
100854,69903,2019-12-25,1,Terno Liso,5,720,3600,Iguatemi Esplanada
100894,69928,2019-12-25,1,Meia,1,30,30,Iguatemi Esplanada
100920,69944,2019-12-25,1,Gorro Xadrez,5,85,425,Iguatemi Esplanada


,Sale Code,Date,Store ID,Product,Quantity,Unit Value,Final Value,Store
36,31,2019-01-02,21,Casaco Estampa,3,256,768,Shopping Eldorado
37,31,2019-01-02,21,Pulseira,3,70,210,Shopping Eldorado
38,31,2019-01-02,21,Gorro Linho,1,124,124,Shopping Eldorado
39,31,2019-01-02,21,Cinto Linho,3,248,744,Shopping Eldorado
51,40,2019-01-02,21,Bermuda Linho,1,195,195,Shopping Eldorado
...,...,...,...,...,...,...,...,...
100935,69959,2019-12-25,21,Polo Liso,4,137,548,Shopping Eldorado
100936,69959,2019-12-25,21,Camisa Linho,3,141,423,Shopping Eldorado
100937,69959,2019-12-25,21,Cueca Xadrez,1,60,60,Shopping Eldorado
100985,69986,2019-12-26,21,Cueca Estampa,4,66,264,Shopping Eldorado


In [6]:
# Find the latest day available and get its data

indicator_day = sales['Date'].max()
print(indicator_day)

2019-12-26 00:00:00


### Step 3 - Save the sheets in the Backup folder


In [12]:
# identify if folder already exists
backup_path = pl.Path(r'Backup')
backup_folder_files = backup_path.iterdir()
backup_name_list = []

for file in backup_folder_files:
    backup_name_list.append(file.name)

In [14]:
# save inside the respective folder
for store in store_dictionary:
    if store not in backup_name_list:
        new_folder = backup_path / store
        new_folder.mkdir()

    file_name = '{}_{}_{}.xlsx'.format(indicator_day.month, indicator_day.day, store)

    file_path = backup_path / store / file_name

    store_dictionary[store].to_excel(file_path)


### Step 4 - Calculate the indicator for one store


### Step 5 - Send email to the store manager


### Step 6 - Automate every store


### Step 7 - Create ranking for the director of the company


### Step 8 - Send email to the director